In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, BertForSequenceClassification, BertTokenizer
import torch

In [2]:
# tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")
# model = BertForSequenceClassification.from_pretrained("mental/mental-bert-base-uncased")

In [3]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

In [4]:
tokenizer = AutoTokenizer.from_pretrained("mental/mental-roberta-base")
model = AutoModel.from_pretrained("mental/mental-roberta-base")

Some weights of the model checkpoint at mental/mental-roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at mental/mental-roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# # Code from the hugging face tutorial
# def print_encoding(model_inputs, indent=4):
#     indent_str = " " * indent
#     print("{")
#     for k, v in model_inputs.items():
#         print(indent_str + k + ":")
#         print(indent_str + indent_str + str(v))
#     print("}")

In [6]:
# inputs = "I am feeling fantastic!"
# tokenized_inputs = tokenizer(inputs, return_tensors="pt")
# outputs = model(**tokenized_inputs)

# labels = ['NEGATIVE', 'POSITIVE']
# prediction = torch.argmax(outputs.logits)


# print("Input:")
# print(inputs)
# print()
# print("Tokenized Inputs:")
# print_encoding(tokenized_inputs)
# print()
# print("Model Outputs:")
# print(outputs)
# print()
# print(f"The prediction is {labels[prediction]}")

In [7]:
from datasets import load_dataset, DatasetDict

In [8]:
# For cleaning

# import csv
# with open('total_labeled_nonan.csv', 'w', newline='') as writefile:
#     with open('total_labeled_data.csv', newline='') as readfile:
#         reader = csv.DictReader(readfile)
#         writer = csv.writer(writefile, delimiter=',')
#         writer.writerow(['label', 'text'])
#         index = 0
#         for row in reader:
#             if row['post'] != 'nan' and row['post'] != '':
#                 writer.writerow([index, row['labels'], row['post']])
#                 index += 1
            

In [9]:
# Load in reddit data
# dataset = load_dataset('csv', data_files='total_labeled_nonan.csv')

In [10]:
# Using snorkel data instead
dataset = load_dataset('csv', data_files='firstPassSnorkelLabels.csv')

Using custom data configuration default-8af1dd697c287706
Reusing dataset csv (/home/basketbla/.cache/huggingface/datasets/csv/default-8af1dd697c287706/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
# for x in range(len(dataset['train']['text'])):
#     if dataset['train']['text'][x] is None:
#         print(x)

In [12]:
# Take 10000 random examples for train and 2000 validation and 2000 test
# dataset = DatasetDict(
#     train=dataset['train'].shuffle(seed=1111).select(range(10000)),
#     val=dataset['train'].shuffle(seed=1111).select(range(10000, 12000)),
#     test=dataset['train'].shuffle(seed=1111).select(range(12000, 14000)),
# )

In [13]:
# For snorkel data: 10000, 1000, 1000 instead
dataset = DatasetDict(
    train=dataset['train'].shuffle(seed=1111).select(range(10000)),
    val=dataset['train'].shuffle(seed=1111).select(range(10000, 11000)),
    test=dataset['train'].shuffle(seed=1111).select(range(11000, 12000)),
)

Loading cached shuffled indices for dataset at /home/basketbla/.cache/huggingface/datasets/csv/default-8af1dd697c287706/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-d35a1bcdd73616e6.arrow
Loading cached shuffled indices for dataset at /home/basketbla/.cache/huggingface/datasets/csv/default-8af1dd697c287706/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-d35a1bcdd73616e6.arrow
Loading cached shuffled indices for dataset at /home/basketbla/.cache/huggingface/datasets/csv/default-8af1dd697c287706/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-d35a1bcdd73616e6.arrow


In [14]:
# for x in range(len(dataset['train']['text'])):
#     if dataset['train']['text'][x] is None:
#         print(x)

In [15]:
BATCH_SIZE = 16

In [16]:
small_tokenized_dataset = dataset.map(
    lambda example: tokenizer(example['text'], padding=True, truncation=True),
    batched=True,
    batch_size=BATCH_SIZE
)

small_tokenized_dataset = small_tokenized_dataset.remove_columns(["text"])
# Sometimes 'labels' doesn't exist?
# small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")
small_tokenized_dataset.set_format("torch")

Loading cached processed dataset at /home/basketbla/.cache/huggingface/datasets/csv/default-8af1dd697c287706/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-2f746465b7e0ac9b.arrow


  0%|          | 0/63 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/basketbla/.cache/huggingface/datasets/csv/default-8af1dd697c287706/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-a3fd2fb90752fae6.arrow


In [17]:
# Tokenize the data
# HUGGING FACE ONE, NOT USING

# def preprocess_function(examples):
#     return tokenizer(examples["text"], truncation=True)

# tokenized_dataset = dataset.map(preprocess_function, batched=True)
# tokenized_train = dataset["train"].map(preprocess_function, batched=True)
# tokenized_val = dataset["val"].map(preprocess_function, batched=True)

In [18]:
# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# data_collator(tokenized_dataset)

In [19]:
# Needed for bert base

# small_tokenized_dataset = small_tokenized_dataset.remove_columns(['__index_level_0__'])
# small_tokenized_dataset = small_tokenized_dataset.remove_columns(['token_type_ids'])

In [20]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_tokenized_dataset['train'], batch_size=BATCH_SIZE)
eval_dataloader = DataLoader(small_tokenized_dataset['val'], batch_size=BATCH_SIZE)

In [21]:
# from transformers import AdamW, get_linear_schedule_with_warmup
# from tqdm.notebook import tqdm


# model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# model.cuda()
# num_epochs = 3
# num_training_steps = 3 * len(train_dataloader)
# optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
# lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# best_val_loss = float("inf")
# progress_bar = tqdm(range(num_training_steps))
# for epoch in range(num_epochs):
#     # training
#     model.train()
#     for batch_i, batch in enumerate(train_dataloader):
#         batch = {k: v.cuda() for k, v in batch.items()}
#         output = model(**batch)
        
# #         model(batch['input_ids'].cuda(),
# #               )
        
#         optimizer.zero_grad()
#         output.loss.backward()
#         optimizer.step()
#         lr_scheduler.step()
#         progress_bar.update(1)
    
#     # validation
#     model.eval()
#     val_loss = 0
#     for batch_i, batch in enumerate(eval_dataloader):
#         batch = {k: v.cuda() for k, v in batch.items()}
#         with torch.no_grad():
#             output = model(**batch)
#         val_loss += output.loss.cpu().detach()
    
#     avg_val_loss = val_loss / len(eval_dataloader)
#     print(f"Validation loss: {avg_val_loss}")
#     if avg_val_loss < best_val_loss:
#         print("Saving checkpoint!")
#         best_val_loss = avg_val_loss
#         torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'val_loss': best_val_loss,
#             },
#             f"checkpoints/epoch_{epoch}.pt"
#         )  

In [22]:
# results = model.predict(small_tokenized_dataset['val'])

In [23]:
# Trying trainer class instead

from transformers import TrainingArguments, Trainer
import numpy as np

arguments = TrainingArguments(
    output_dir="mental_roberta_snorkel_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224
)


def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [24]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaModel.forward` and have been ignored: labels.
/anaconda/envs/py38_default/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1875


KeyError: 'loss'

In [ ]:
# import mlflow
# mlflow.end_run()

In [ ]:
trainer.train("mental_roberta_snorkel_trainer/checkpoint-1875")

In [ ]:
# Do predictions on test set
results = trainer.predict(small_tokenized_dataset['test'])

In [ ]:
# Compute metrics (accuracy, F1, whatever else)
from datasets import load_metric

preds = [0 if pair[0] > pair[1] else 1 for pair in results.predictions]

metric = load_metric("accuracy")
metric.add_batch(predictions=preds, references=dataset['test']['labels'])
final_score = metric.compute()
print(final_score)

metric = load_metric("f1")
metric.add_batch(predictions=preds, references=dataset['test']['labels'])
final_score = metric.compute()
print(final_score)

metric = load_metric("recall")
metric.add_batch(predictions=preds, references=dataset['test']['labels'])
final_score = metric.compute()
print(final_score)

